<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Julia_prog_language.svg/220px-Julia_prog_language.svg.png" alt="drawing" width="180"/>

## [julialang.org](https://julialang.org/)

* Fairly new, first stable 1.0 release in 2018.
* Interpreted language, syntax should feel familiar to Python and MATLAB users.
* Uses a precompiler to speed up repeated function calls (good when formulating big models).

<img src="https://camo.githubusercontent.com/31d60f762b44d0c3ea47cc16b785e042104a6e03/68747470733a2f2f7777772e6a756c69616f70742e6f72672f696d616765732f6a756d702d6c6f676f2d776974682d746578742e737667" alt="drawing" width="300"/>

## [JuMP.jl](http://www.juliaopt.org/JuMP.jl/v0.18/)

* Provides an abstraction for building MIP models in a solver independent way.
* Reasonably fast (once the precompiler gets up to speed).
* Mapping between MIP model mathematical notation and code is basically one to one.
* Still under development (last release made some big changes - using the previous version here).

<img src="https://juliabox.com/assets/img/juliabox-big.svg" alt="JuliaBox" width="220"/>

## [juliabox.com](https://juliabox.com/#)

* Julia code runs online, presents a cell-based IDE.
* Very useful for teaching (gets everyone in the same environment).
* Uses [Jupyter](https://jupyter.org) in the background.

# MIP Modelling Requirements

* Julia's package manager is pretty straightforward.
* In this case we just need JuMP and the GLPK MIP solver.
* JuMP allows us to replace GLPK with any suitable solver

    * Commercial - Gurobi, CPLEX
    * Commercial-ish (?) - SCIP
    * Free - GLPK, Clp/Cbc, ...

In [1]:
# Fix versions of the packages we need to ensure JuliaBox compatibility.
import Pkg

Pkg.add([
    Pkg.PackageSpec(name="JuMP", version="0.18.5"),
    Pkg.PackageSpec(name="GLPKMathProgInterface", version="0.4.4")
])

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [2]:
import Pkg
Pkg.add("JuMP")                    # Modelling components.
Pkg.add("GLPKMathProgInterface")   # Backend solver.

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


# Initialising a Model

This is the only solver-specific part.

In [3]:
using JuMP
using GLPKMathProgInterface

model = Model(solver=GLPKSolverMIP())

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

# Part 1: Generating Feasible Crop Schedules

Our model defines feasible crop rotation schedules on a single plot.

Technical criteria:
* Each crop $i$ can only be grown in a subset $I_i$ of planting periods.
* Each crop has a specific growing cycle ( $t_i$ ).

Ecological-based criteria:
* Crops of the same botanical family cannot be grown one immediately after one another.
* Green manuring crops must be planted during the cycle.
* A fallow period is required.

\begin{alignat*}{2}
\text{No overlap:} \quad\quad & \sum_{i=1}^{N+1} \sum_{r=0}^{t_i-1} x_{i,j-r} \le 1, \quad j = 1 \dots M, \\
\text{Separate siblings:} \quad\quad & \sum_{i \in F_p} \sum_{r=0}^{t_i} x_{i,j-r} \le 1, \quad p = 1 \dots |F_p|,\, j = 1 \dots M, \\
\text{Plant green manure crop:} \quad\quad & \sum_{i\in G} \sum_{j \in I_i} x_{i,j} \le 1, \\
\text{Require a fallow period:} \quad\quad & \sum_{j=1}^{M} x_{N+1,j} = 1, \\
\text{Valid planting times:} \quad\quad & x_{ij} \in \lbrace 0, 1 \rbrace, \quad i = 1 \dots N + 1,\, j = \in I_i  \\
\end{alignat*}

## Defining Problem Data

* Problem data is stored in native Julia arrays.
* Any data structure can be used in the formulation.

In [4]:
# Crop rotation schedule data.
M = 12             # Number of periods considered in the 12 month rotation.
N = 3              # Number of crops.
n = N+1            # Fallow 'crop'.
G = [3]            # Set of green manuring crops.
F = [[1, 2], [3]]  # Set of crop families.
t = [5, 4, 2, 1]   # Production time (months) for each crop.
I = [
    1:4,              # Crop 1 may be planted between January and April
    1:12,             # Crop 2 may be planted any time.
    vcat(1:3, 7:12),  # Crop 3 may be planted between July and March
    1:12              # Fallow period (any time).
];

## Defining Variables: valid planting times

* $I_i$ is the set of periods when crop $i$ can be planted.
* We define variables $x_{ij}$ only for valid planting times $j$ of crop $i$.
* $x_{ij} = 1$ indicates that crop $i$ is planted in period $j$ of the rotation.

In [5]:
model = Model(solver=GLPKSolverMIP())
@variable(model, x[i in 1:N+1, j in I[i]], Bin)  # Yes, we can nest indices!

x[i,j] ∈ {0,1} ∀ i ∈ {1,2,3,4}, j ∈ {…}

### One period of fallow required.

* Recall $n = N+1 = 4$ represents a dummy crop enforcing fallow.

$$\sum_{j=1}^{M} x_{N+1,j} = 1$$

In [6]:
@constraint(model, fallow, sum(x[N + 1, j] for j in 1:M) == 1)

x[4,1] + x[4,2] + x[4,3] + x[4,4] + x[4,5] + x[4,6] + x[4,7] + x[4,8] + x[4,9] + x[4,10] + x[4,11] + x[4,12] = 1

### One green manure crop required.

* Recall crop 3 may be planted between July and March (months 7 -> 3).
* The set $G$ lists manure crops.

$$\sum_{i\in G} \sum_{j \in I_i} x_{i,j} \le 1$$

In [7]:
@constraint(model, green_manure,
    sum(
        sum(x[i, j] for j in I[i])
        for i in G) == 1)

x[3,1] + x[3,2] + x[3,3] + x[3,7] + x[3,8] + x[3,9] + x[3,10] + x[3,11] + x[3,12] = 1

## Constraint: No overlap of crop plantings

$$\sum_{i=1}^{N+1} \sum_{r=0}^{t_i-1} x_{i,j-r} \le 1, \quad j = 1 \dots M$$

We can model this constraint for a single time period and inspect the result:

In [8]:
j = 3    # Generate this constraint for March.
@constraint(model,
    sum(
        sum(
            x[i, mod(j - r - 1, M) + 1]         # Wrap around.
            for r in 0:t[i]-1                   # Look back over the production time.
            if mod(j - r - 1, M) + 1 in I[i])   # Only consider valid planting times.
        for i in 1:N+1) <= 1)

x[1,3] + x[1,2] + x[1,1] + x[2,3] + x[2,2] + x[2,1] + x[2,12] + x[3,3] + x[3,2] + x[4,3] ≤ 1

The following code generates these constraints for every planting period:

In [9]:
@constraint(model,
    no_overlap[j in 1:M],
    sum(
        sum(
            x[i, mod(j - r - 1, M) + 1]
            for r in 0:t[i]-1
            if mod(j - r - 1, M) + 1 in I[i])
        for i in 1:N+1) <= 1);

### Crops of the same family may not be planted after one another.

$$\sum_{i \in F_p} \sum_{r=0}^{t_i} x_{i,j-r} \le 1, \quad p = 1 \dots |F_p|,\, j = 1 \dots M$$

Inspect the expression generated for period 1 and family 1 (containing production crops 1 and 2).

In [10]:
f = F[1]
j = 4
@constraint(model,
    sum(
        sum(
            x[i, mod(j - r - 1, M) + 1]
            for r in 0:t[i]       # Looks back further to enforce a break.
            if mod(j - r - 1, M) + 1 in I[i])
        for i in f) <= 1)    # Only applies to crops in the same family.

x[1,4] + x[1,3] + x[1,2] + x[1,1] + x[2,4] + x[2,3] + x[2,2] + x[2,1] + x[2,12] ≤ 1

This constraint is added to the model for every planting period and every crop family:

In [11]:
@constraint(model,
    sibling_rivalry[f in F, j in 1:M],
    sum(
        sum(
            x[i, mod(j - r - 1, M) + 1]
            for r in 0:t[i]
            if mod(j - r - 1, M) + 1 in I[i])
        for i in f) <= 1);

## Set the objective function.

* In this case we assign a simple objective function to maximise utilisation.

$$\sum_{i=1}^{N} \sum_{j \in I_i} t_i x_{ij}$$

In [12]:
@objective(model, Max,
    sum(
        sum(
            t[i] * x[i, j]
            for j in I[i])
        for i in 1:N))

5 x[1,1] + 5 x[1,2] + 5 x[1,3] + 5 x[1,4] + 4 x[2,1] + 4 x[2,2] + 4 x[2,3] + 4 x[2,4] + 4 x[2,5] + 4 x[2,6] + 4 x[2,7] + 4 x[2,8] + 4 x[2,9] + 4 x[2,10] + 4 x[2,11] + 4 x[2,12] + 2 x[3,1] + 2 x[3,2] + 2 x[3,3] + 2 x[3,7] + 2 x[3,8] + 2 x[3,9] + 2 x[3,10] + 2 x[3,11] + 2 x[3,12]

## Solving and Inspecting

Model-and-solve paradigm is easy!

In [13]:
status = solve(model)
println("Status: ", status)
println("Months of Land Use: ", Int(round(getobjectivevalue(model))))

Status: Optimal
Months of Land Use: 11


In [14]:
getvalue(x)

x: 2 dimensions, 37 entries:
 [1, 1] = 0.0
 [1, 2] = 1.0
 [1, 3] = 0.0
 [1, 4] = 0.0
 [2, 1] = 0.0
 [2, 2] = 0.0
 [2, 3] = 0.0
 [2, 4] = 0.0
 [2, 5] = 0.0
 [2, 6] = 0.0
 [2, 7] = 0.0
 [2, 8] = 1.0
 [2, 9] = 0.0
 [2,10] = 0.0
 [2,11] = 0.0
 [2,12] = 0.0
 [3, 1] = 0.0
 [3, 2] = 0.0
 [3, 3] = 0.0
 [3, 7] = 0.0
 [3, 8] = 0.0
 [3, 9] = 0.0
 [3,10] = 0.0
 [3,11] = 0.0
 [3,12] = 1.0
 [4, 1] = 0.0
 [4, 2] = 0.0
 [4, 3] = 0.0
 [4, 4] = 0.0
 [4, 5] = 0.0
 [4, 6] = 0.0
 [4, 7] = 1.0
 [4, 8] = 0.0
 [4, 9] = 0.0
 [4,10] = 0.0
 [4,11] = 0.0
 [4,12] = 0.0

## Producing readable plans

In [15]:
month = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]
crop = ["A", "B", "C"]

for j in 1:M
    for i in 1:N
        if j in I[i] && getvalue(x[i,j]) == 1
            j_end = j + t[i] - 1
            println("Plant ", crop[i], " in ", month[j], ", harvest in ", month[mod(j_end - 1, M) + 1])
        end
    end
end

Plant A in February, harvest in June
Plant B in August, harvest in November
Plant C in December, harvest in January


## Displaying schedules

In [16]:
function get_action(j, x, M, N)
    plant = [i for i in 1:N if j in I[i] && getvalue(x[i, j]) == 1]
    if !isempty(plant)
        return first(plant)
    else
        harvest = [
            i for i in 1:N for r in 1:(t[i] - 1)
            if mod(j - r - 1, M) + 1 in I[i]
            && getvalue(x[i, mod(j - r - 1, M) + 1]) == 1]
        if length(harvest) > 0
            return "-"
        end
    end
    return " "
end

function get_schedule(x, M, N)
    "|" * join([get_action(j, x, M, N) for j in 1:M], "|") * "|"
end

println("Schedule:  ", get_schedule(x, M, N))

Schedule:  |-|1|-|-|-|-| |2|-|-|-|3|


# An Alternative Approach

* Above shows an easy translation of a MIP formulation to JuMP.
* We can also be a bit more flexible and write a more readable model.

In [17]:
months = 12
crops = ["A", "B", "C"]
manure_crops = ["C"]
crop_families = [["A", "B"], ["C"]]
production_time = Dict("A" => 5, "B" => 4, "C" => 2)
planting_times = Dict(
    "A" => 1:4,
    "B" => 1:12,
    "C" => vcat(1:3, 7:12))

fallow = "F"
all_crops = vcat(crops, [fallow])
planting_times["F"] = 1:months
production_time["F"] = 1;

In [20]:
model = Model(solver=GLPKSolverMIP())
@variable(model, plant[crop in all_crops, month in planting_times[crop]], Bin)
@constraint(model, c_fallow, sum(plant[fallow, month] for month in planting_times["F"]) == 1)
@constraint(model, c_manure_crops,
    sum(
        sum(plant[crop, month] for month in planting_times[crop])
        for crop in manure_crops) == 1)
@constraint(model,
    no_overlap[month in 1:M],
    sum(
        sum(
            plant[crop, mod(month - r - 1, months) + 1]
            for r in 0:(production_time[crop]-1)
            if mod(month - r - 1, months) + 1 in planting_times[crop])
        for crop in all_crops) <= 1);
@constraint(model,
    sibling_rivalry[family in crop_families, month in 1:months],
    sum(
        sum(
            plant[crop, mod(month - r - 1, M) + 1]
            for r in 0:production_time[crop]
            if mod(month - r - 1, M) + 1 in planting_times[crop])
        for crop in family) <= 1)
@objective(model, Max,
    sum(
        production_time[crop] * plant[crop,month]
        for crop in crops
        for month in planting_times[crop]));